# Q-Polynomial of ...

First, we initialize the matroid and obtain the lattice of flats. We relabel the flats for simple presentation.

In [192]:
r, n = 5, 6
matroid = matroids.Uniform(r, n)
graded_flats = [list(matroid.flats(i)) for i in range(n)]
flats = sum(graded_flats, [])  # Flatten the list of flats

In [ ]:
#plottting the geometric lattice:
#size = 0 for now

labels = {element: idx for idx, element in enumerate(flats)}
matroid.lattice_of_flats().plot(
    element_labels = labels, element_color = "white",
    figsize= 0, cover_color = "grey")

Graphics object consisting of 230 graphics primitives

In [ ]:
R.<t> = PolynomialRing(QQ)

def the_set_S(flats, e):
    return set(F for F in flats if e.isdisjoint(F) and F.union(e) in flats)

def gx_uniformQpoly(r, n):
    if r == n : return 1
    m, d = n-r, r
    qpoly = R(0)
    for j in range (0, (d-1)//2 + 1):
        coeff = m * (d - 2 * j)/((m + j) * (m + d - j)) * (binomial(d, j))
        qpoly = qpoly + coeff * t**(j)
    return binomial(m+d, d) * qpoly

def glxyz_uniformPpolys(r, n):
    if r == n : return R(1)
    m, d = n-r, r
    ppoly = R(0)
    for i in range (0, (d-1)//2 + 1):
        prod = binomial(m + d, i)/(d - i)
        the_sum = sum([binomial(d-i+h, h+i+1)*binomial(i-1+h, h) for h in range (0, m)])
        ppoly = ppoly + prod * the_sum * t ** (i)
    return ppoly

def uniformQWrapper(matroid):
    return gx_uniformQpoly(matroid.rank(), len(matroid.groundset()))

def uniformPWrapper(matroid):
    return glxyz_uniformPpolys(matroid.rank(), len(matroid.groundset()))

def bv_tau_function(matroid, S, e):
    """
    Computes the bv_tau_function for a given matroid and set S.

    Args:
        matroid: The matroid object.
        S: A collection of subsets (typically sets or frozensets).
        e: An element to be unioned with each subset in S.

    Returns:
        A list of values computed for each subset in S.
    """
    return [
        0 if (contraction := matroid.contract(f.union(e)).simplify()).rank() % 2 == 0
        else uniformPWrapper(contraction).coefficients(sparse = False)[-1]
        for f in S
    ]

def bv_delform_rhs(matroid, e, func):
    """
    Computes the right-hand side of the BV deletion-contraction formula.

    Args:
        matroid: The matroid object.
        flats: The collection of flats of the matroid.
        e: The element being deleted/contracted.
        func: A function to apply to matroids.

    Returns:
        The computed polynomial representing the right-hand side.
    """
    rank = matroid.rank()

    deletion = matroid.delete(e).simplify()
    contraction = matroid.contract(e).simplify()

    set_S = list(the_set_S(flats, e))

    # Compute bv_tau_function and zip with set_S
    bv_tau_values = bv_tau_function(matroid, set_S, e)

    # Compute the summation over S
    sum_S = sum(
        x * t ** ((rank - matroid.rank(y))//2) * func(matroid.delete(matroid.groundset() - y).simplify())
        for x, y in zip(bv_tau_values, set_S)
    )
    # Compute the RHS
    return func(deletion) - t * func(contraction) + sum_S

print(uniformQWrapper(matroid))
bv_delform_rhs(matroid, flats[1], uniformQWrapper)

5*t^2 + 9*t + 5


5.0*t^2 + 6.0*t + 1.0

Q-polynomials for parallel connection of cycle graphs will be implemented below: